# 2021_05_03 Delay Analysis

Notebook to perform analysis on delay period (2 or 4 seconds) of PWM task recording from mPFC. Functions tested & debugged in event_centering.ipynb and moving here for clean start

### Libraries & Settings

In [14]:
# libraries
import sys; sys.path.insert(0, '..') # if you don't find it here, look one above
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy.io as spio
import pickle
import warnings

from tqdm import tqdm
from scipy import stats
from scipy.ndimage import gaussian_filter1d
from io_utils import *
from plotting_utils import * 


# settings
%load_ext autoreload
%autoreload 2
sns.set_context("talk")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Set paths & defaults

In [34]:
### Set paths & defaults

sess_names = np.array(['data_sdb_20190724_193007_fromSD', 'data_sdc_20190902_145404_fromSD', 'W122_07_27_2019_1_fromSD', 
            'W122_08_06_2019_1_fromSD', 'W122_08_07_2019_1_fromSD', 'W122_08_31_2019_1_fromSD', 'W122_09_04_2019_1_fromSD',
            'W122_09_09_2019_1_fromSD', 'W122_09_12_2019_1_fromSD','W122_09_21_2019_1_fromSD', 'W122_09_24_2019_1_fromSD'])

base_path  = 'Y:\jbreda\ephys\post_sort_analysis\sorted_pre_bdata' 
beh_mat   = 'protocol_info.mat'
spks_mat  = 'ksphy_clusters_foranalysis.mat'

fig_save_path = os.path.join(os.getcwd(), 'figures', 'delay_loudness')

# create paths
sess_paths = [os.path.join(base_path, sess) for sess in sess_names]
beh_path  = [os.path.join(sess_path, beh_mat) for path in sess_path]
spks_path = [os.path.join(sess_path, spks_mat) for path in sess_path]

# alginment info (hard coded here & in fx)
names = ['aud1on', 'aud1off', 'aud2on', 'aud2off', 'trial_all', 
         'delay2s', 'trial2s', 'delay4s', 'trial4s']
windows = [[-400, 600], [-500, 500], [-400, 600], [-500, 500], [-100, 5000],
           [-150, 2150],[-200, 2400], [-150, 4150], [-200, 4400]]

align_windows = dict(zip(names,windows))

events = ['delay2s', 'delay4s']

### Load in data & align spikes to events

I'm filtering the dataframe below specifically for delay analysis (no psychometric sounds, no violations, no 6 second trials)

In [35]:
for name, path, behp, spksp in zip(sess_names, sess_paths, beh_path, spks_path):
    
    print(f"Loading session {name}")
    
    # LOAD
    beh_df, spks_dict = load_and_wrangle(behp, spksp, overwrite=False)
    
    delay_analysis_df = beh_df[(beh_df['delay'] != 6) & (beh_df['first_sound'] != 'psycho') &
                                     (beh_df['hit_hist'] != 'viol')]
    
    del_2_df = delay_analysis_df[(delay_analysis_df['delay'] == 2)] 
    del_4_df =delay_analysis_df[(delay_analysis_df['delay'] == 4)]
    dfs = [del_2_df, del_4_df]
    
    print("Aligning to events")
    
    sess_aligned = event_align_session(spks_dict, delay_analysis_df, path, overwrite=False, 
                                   file_name='delay_analysis_aligned_spks.pkl')
    
    print("Plotting")
    
    run_delay_analysis_for_session(name, sess_aligned, align_windows, events, 
                                   dfs, fig_save_path, path)
    

Loading session data_sdb_20190724_193007_fromSD
Aligning to events
no file found, running alignment for session
Plotting
Loading session data_sdc_20190902_145404_fromSD
Aligning to events
no file found, running alignment for session
Plotting
Loading session W122_07_27_2019_1_fromSD
Aligning to events
no file found, running alignment for session
Plotting
Loading session W122_08_06_2019_1_fromSD
Aligning to events
loading from file...
Plotting
Loading session W122_08_07_2019_1_fromSD
Aligning to events
no file found, running alignment for session
Plotting
Loading session W122_08_31_2019_1_fromSD
Aligning to events
no file found, running alignment for session
Plotting
Loading session W122_09_04_2019_1_fromSD
Aligning to events
no file found, running alignment for session
Plotting
Loading session W122_09_09_2019_1_fromSD
Aligning to events
no file found, running alignment for session
Plotting
Loading session W122_09_12_2019_1_fromSD
Aligning to events
no file found, running alignment for s

In [4]:
beh_df, spks_dict = load_and_wrangle(beh_path, spks_path, overwrite=False)

delay_analysis_df = beh_df[(beh_df['delay'] != 6) & (beh_df['first_sound'] != 'psycho') &
                                     (beh_df['hit_hist'] != 'viol')]

sess_aligned = event_align_session(spks_dict, delay_analysis_df, sess_path, overwrite=False, 
                                   file_name='delay_analysis_aligned_spks.pkl')

loading from file...


In [5]:
# choose events to analyze & filter dfs for them
events = ['delay2s', 'delay4s']


del_2_df = delay_analysis_df[(delay_analysis_df['delay'] == 2)] 
del_4_df =delay_analysis_df[(delay_analysis_df['delay'] == 4)]
dfs = [del_2_df, del_4_df]

In [6]:
def analyze_and_plot_loudness(sess_name, sess_aligned, aligned_windows, event, df,
                              fig_save_path, sess_path):
    
    # initialize lists for saving out
    trials_loudness = []
    summary_stats = []
    
    for neuron in range(len(sess_aligned)):
        
        neuron_id = sess_name + '_N' + str(neuron)
        
        # calculate psth via gaussian and boxcar
        psth_g = PSTH_gaussain(sess_aligned[neuron], align_windows, event, df, 
                               conditions='first_sound', sigma=150)
        psth_b = PSTH_boxcar(sess_aligned[neuron], align_windows, event, df,
                             conditions='first_sound', bin_size=0.150)

        ## PSTHs
        fig, ax = plt.subplots(2,1, figsize = (15,12))
        plot_psth(psth_g, ax[0], legend=True, title=neuron_id)
        plot_psth(psth_b, ax[1])

        # save out
        fig_name = f"{neuron_id}_{event}_psth"
        plt.tight_layout()
        plt.savefig(os.path.join(fig_save_path, fig_name))
        plt.close("all") 
        
        # FIRING RATE ~ LOUDNESS
        # extract data
        loudness_df = fr_by_loudness_df(psth_g, neuron_id)
        trials_loudness.append(loudness_df)

        # plot & regress
        fig2,ax2 = plt.subplots(1,1, figsize=(8,5))
        regression_stats = regress_loudness_and_plot(loudness_df, ax=ax2)
        summary_stats.append(regression_stats)

        # save out
        fig_name = f"{neuron_id}_{event}_loudness"
        plt.tight_layout()
        plt.savefig(os.path.join(fig_save_path, fig_name))
        plt.close("all") 

    
    # concat & save out data frames used for regression
    stats_df = pd.concat(summary_stats)
    stats_df.reset_index().to_csv(os.path.join(sess_path, 'fr_by_loudness.csv'))

    loudness_df = pd.concat(trials_loudness)
    loudness_df.reset_index().to_csv(os.path.join(sess_path, 'fr_by_loudness_regression.csv'))
        

In [17]:
def run_delay_analysis_for_session(sess_name, sess_aligned, aligned_windows, events, dfs, 
                       fig_save_path, sess_path):
    
    for event, df in zip(events, dfs):
        
        analyze_and_plot_loudness(sess_name, sess_aligned, align_windows,
                                  event, df, fig_save_path, sess_path)


In [15]:
run_delay_analysis_for_session(sess_name, sess_aligned, align_windows, events, dfs, fig_save_path, sess_path)